In [90]:
import joblib
import function_library as fl
import importlib
import pandas as pd
import time
importlib.reload(fl)

<module 'function_library' from 'C:\\Users\\leere\\PycharmProjects\\Football_ML3\\function_library.py'>

In [91]:
file_path = r"C:\Users\leere\OneDrive\Desktop\RAW DATA\GOALS_LIVE.CSV"

# List of common encodings to try
encodings = ['utf-8', 'latin1', 'iso-8859-1', 'cp1252']

data = None
for encoding in encodings:
    try:
        data = pd.read_csv(file_path, encoding=encoding)
        print(f"Successfully read the file with encoding: {encoding}")
        break
    except UnicodeDecodeError:
        print(f"Failed to decode with encoding: {encoding}")

col_dict = {
    "country": "Country",
    "league": "League",
    "datameci": "Date",
    "etapa": "Round",
    "txtechipa1": "home_team",
    "txtechipa2": "away_team",
    "place1t": "Home_team_place_total",
    "place1a": "Home_team_place_home",
    "place2t": "Away_team_place_total",
    "place2d": "Away_team_place_away",
    "customh": "ELO_home",
    "customa": "ELO_away",
    "custom3": "FORM_home",
    "custom4": "FORM_away",
    "home_val": "home_win",
    "home_val_2": "home_win_15",
    "home_val_3": "home_o25",
    "home_val_4": "home_o35",
    "home_val_5": "home_scored",
    "away_val": "away_win",
    "away_val_2": "away_win_15",
    "away_val_3": "away_o25",
    "away_val_4": "away_o35",
    "away_val_5": "away_scored",
    "scor1": "home_goals",
    "scor2": "away_goals",
    "cotao": "o2.5_odds",
}

data = data.rename(columns=col_dict).filter(items=col_dict.values())
# Convert Date column to datetime type
data['Date'] = pd.to_datetime(data['Date'], format='%d/%m/%Y')

# Order by date
data = data.sort_values(by='Date')

data = data[data["Round"] >= 8]
data.dropna(inplace=True)
data.reset_index(drop=True, inplace=True)

cols_to_drop = ['Date','home_team', 'away_team', 'home_goals', 'away_goals', 'o2.5_odds']

data_ready =data.drop(columns=cols_to_drop)
data_ready

Successfully read the file with encoding: utf-8


,Country,League,Round,Home_team_place_total,Home_team_place_home,Away_team_place_total,Away_team_place_away,ELO_home,ELO_away,FORM_home,...,home_win,home_win_15,home_o25,home_o35,home_scored,away_win,away_win_15,away_o25,away_o35,away_scored
0,Netherlands,Eerste Divisie,26,14,15,19,14,1447.0,1270.0,5.0,...,33.333,25.000,83.333,66.667,2.333,30.769,30.769,46.154,38.462,1.231
1,Poland,Ekstraklasa,21,14,14,17,14,1830.0,1793.0,-13.0,...,44.444,11.111,55.556,22.222,1.111,10.000,0.000,60.000,50.000,1.300
2,Portugal,Primeira L,22,14,10,9,7,1750.0,1793.0,-6.0,...,40.000,30.000,60.000,20.000,1.600,30.000,20.000,60.000,30.000,1.500
3,Romania,Liga 1,27,6,3,14,14,2016.0,1825.0,-1.0,...,53.846,23.077,38.462,23.077,1.615,15.385,7.692,46.154,15.385,0.615
4,Romania,Liga 1,27,4,2,13,12,2009.0,1791.0,-3.0,...,61.538,30.769,61.538,30.769,1.923,30.769,7.692,53.846,7.692,0.846
5,Slovenia,Prvaliga,21,8,8,10,9,1822.0,1710.0,-9.0,...,30.000,20.000,30.000,20.000,1.300,10.000,0.000,70.000,40.000,0.800
6,Turkey,Super Lig,24,11,13,1,1,1879.0,2386.0,-4.0,...,40.000,20.000,50.000,30.000,1.400,90.000,40.000,70.000,40.000,2.600
7,Turkey,1. Lig,25,19,19,10,13,1576.0,1744.0,0.0,...,27.273,9.091,45.455,27.273,1.182,33.333,16.667,58.333,33.333,1.000
8,Turkey,1. Lig,25,7,7,8,17,1712.0,1733.0,-20.0,...,58.333,25.000,50.000,41.667,1.750,25.000,8.333,58.333,25.000,1.083
9,Turkey,1. Lig,25,2,2,18,18,1878.0,1651.0,3.0,...,66.667,25.000,41.667,41.667,1.917,16.667,8.333,41.667,16.667,0.833


In [92]:
# Directory where models are saved
models_dir = "ALL_MODELS"

# Group unseen data by League and Sub-League
grouped_data = data_ready.groupby(['Country', 'League'])

# Dictionary to store predictions
predictions = {}

# Iterate through each group in the unseen data
for (country, league), group in grouped_data:
    # Construct the model filename
    model_filename = f"{models_dir}/{country}_{league}_model.pkl"

    try:
        # Load the corresponding model
        saved_data = joblib.load(model_filename)  # Load the entire dictionary
        model = saved_data['model']  # Extract the model
        scaler = saved_data['scaler']  # Extract the scaler
        threshold = saved_data['threshold']  # Extract the threshold

        # Extract features for prediction (exclude grouping columns)
        X = group.drop(columns=['Country', 'League'])

        # Scale the features
        X_scaled = scaler.transform(X)

        # Make probability predictions using the model
        y_proba = model.predict_proba(X_scaled)[:, 1]  # Positive class probabilities

        # Filter predictions based on the threshold
        group['Probability'] = y_proba
        group['Prediction'] = (y_proba >= threshold).astype(int)

        # Keep only rows where Prediction == 1
        filtered_predictions = group[group['Prediction'] == 1]

        # Add `home_team` and `away_team` from the original `data`
        game_info = data.loc[filtered_predictions.index, ['home_team', 'away_team', 'Date']]
        filtered_predictions = pd.concat([game_info, filtered_predictions], axis=1)

        # Add filtered predictions to the dictionary
        predictions[(country, league)] = filtered_predictions
        print(f"Filtered predictions made for {country} - {league}")

    except FileNotFoundError:
        print(f"Model not found for {country} - {league}. Skipping...")

    except Exception as e:
        print(f"Error processing {country} - {league}: {e}")

# Combine all predictions into a single DataFrame
if predictions:
    all_predictions = pd.concat(predictions.values(), ignore_index=True)
    print("\nAll Predictions as a DataFrame:")
    print(all_predictions)
else:
    print("No predictions available.")


Filtered predictions made for Bulgaria - Premier L
Filtered predictions made for Denmark - Superligaen
Filtered predictions made for England - Championship
Filtered predictions made for England - League One
Filtered predictions made for England - League Two
Filtered predictions made for England - Premier L
Filtered predictions made for France - Ligue 2
Filtered predictions made for Israel - Premier League
Filtered predictions made for Italy - Serie A
Filtered predictions made for Netherlands - Eerste Divisie
Filtered predictions made for Poland - Ekstraklasa
Filtered predictions made for Portugal - Primeira L
Filtered predictions made for Romania - Liga 1
Filtered predictions made for Scotland - Championship
Filtered predictions made for Slovenia - Prvaliga
Filtered predictions made for Spain - Primera
Filtered predictions made for Spain - Segunda
Model not found for Turkey - 1. Lig. Skipping...
Filtered predictions made for Turkey - Super Lig

All Predictions as a DataFrame:
      hom

In [93]:
#all_predictions

In [94]:
all_predictions = fl.team_name_map(all_predictions)
# Get today's date (normalized to midnight so we only compare dates)

In [95]:
today = pd.Timestamp.today().normalize()
# Filter the DataFrame to include only rows with dates greater than or equal to today
all_predictions = all_predictions[all_predictions['Date'] >= today]
all_predictions

,home_team,away_team,Date,Country,League,Round,Home_team_place_total,Home_team_place_home,Away_team_place_total,Away_team_place_away,...,home_o25,home_o35,home_scored,away_win,away_win_15,away_o25,away_o35,away_scored,Probability,Prediction
0,Randers,FC Copenhagen,2025-02-17,Denmark,Superligaen,18,4,7,1,3,...,50.000,37.500,1.750,37.500,25.000,50.000,25.000,1.500,0.710747,1
1,Shrewsbury,Huddersfield,2025-02-18,England,League One,26,22,19,5,5,...,53.333,26.667,1.200,50.000,28.571,64.286,35.714,1.786,0.602513,1
2,Jong AZ Alkmaar,Jong PSV Eindhoven,2025-02-17,Netherlands,Eerste Divisie,26,14,15,19,14,...,83.333,66.667,2.333,30.769,30.769,46.154,38.462,1.231,0.866667,1
3,Rapid Bucharest,Botosani,2025-02-17,Romania,Liga 1,27,6,3,14,14,...,38.462,23.077,1.615,15.385,7.692,46.154,15.385,0.615,0.563846,1
4,Universitatea Cluj,Unirea Slobozia,2025-02-17,Romania,Liga 1,27,4,2,13,12,...,61.538,30.769,1.923,30.769,7.692,53.846,7.692,0.846,0.550904,1
5,Morton,Hamilton,2025-02-18,Scotland,Championship,20,7,8,8,7,...,30.000,20.000,1.300,18.182,9.091,72.727,45.455,1.273,0.637986,1


In [96]:
fl.create_import_file(all_predictions, r"C:\Users\leere\OneDrive\Desktop\IMPORTS\LAY_U25_ALL_IMPORT.csv", provider="lay_all_u25", market_name="Over/Under 2.5 Goals", selection_name="Under 2.5 Goals")

File created and saved successfully at: C:\Users\leere\OneDrive\Desktop\IMPORTS\LAY_U25_ALL_IMPORT.csv
